In [2]:
# Installing the dependencies
import pandas as pd
import numpy as np

df = pd.read_csv('Book1.csv')
df


,age,income,student,credit rating,buys_computer
0,<=30,high,no,fair,no
1,<=30,high,no,excellent,no
2,31 . . .40,high,no,fair,yes
3,>40,medium,no,fair,yes
4,>40,low,yes,fair,yes
5,>40,low,yes,excellent,no
6,31 . . .40,low,yes,excellent,yes
7,<=30,medium,no,fair,no
8,<=30,low,yes,fair,yes
9,>40,medium,yes,fair,yes


In [4]:
# A1. For the data provided below, calculate the prior probability for each class. 

# Data
data = [
    ['<=30', 'high', 'no', 'fair', 'no'],
    ['<=30', 'high', 'no', 'excellent', 'no'],
    ['31 . . .40', 'high', 'no', 'fair', 'yes'],
    ['>40', 'medium', 'no', 'fair', 'yes'],
    ['>40', 'low', 'yes', 'fair', 'yes'],
    ['>40', 'low', 'yes', 'excellent', 'no'],
    ['31 . . .40', 'low', 'yes', 'excellent', 'yes'],
    ['<=30', 'medium', 'no', 'fair', 'no'],
    ['<=30', 'low', 'yes', 'fair', 'yes'],
    ['>40', 'medium', 'yes', 'fair', 'yes'],
    ['<=30', 'medium', 'yes', 'excellent', 'yes'],
    ['31 . . .40', 'medium', 'no', 'excellent', 'yes'],
    ['31 . . .40', 'high', 'yes', 'fair', 'yes'],
    ['>40', 'medium', 'no', 'excellent', 'no']
]

# Extract the "buys_computer" column
buys_computer_column = [row[-1] for row in data]

# Calculate the prior probabilities
total_instances = len(buys_computer_column)
buys_computer_no = buys_computer_column.count('no')
buys_computer_yes = buys_computer_column.count('yes')

# Calculate the prior probabilities
prior_no = buys_computer_no / total_instances
prior_yes = buys_computer_yes / total_instances

# Print the results
print(f'Prior probability for "buys_computer = no": {prior_no:.3f}')
print(f'Prior probability for "buys_computer = yes": {prior_yes:.3f}')



Prior probability for "buys_computer = no": 0.357
Prior probability for "buys_computer = yes": 0.643
